In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [3]:
#검색할 식당 데이터 임포트
rest_data = pd.read_csv('마포구130.csv', encoding='utf-8-sig')
rest_data

,name
0,비터스
1,무쇠김치삼겹
2,랫댓
3,남산호랭이
4,하이디라오
...,...
97,닝교초식당
98,곱창파는고깃집
99,고미
100,딩가케이크


In [4]:
#결측치 처리 및 식당명만 변수 지정
rest_data=rest_data.dropna()
rest_data.info()

#지역명이 포함된 식당명 컬럼을 변수로 지정
items = rest_data['name']
print(items)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    102 non-null    object
dtypes: object(1)
memory usage: 944.0+ bytes
0           비터스
1        무쇠김치삼겹
2            랫댓
3         남산호랭이
4         하이디라오
         ...   
97        닝교초식당
98      곱창파는고깃집
99           고미
100       딩가케이크
101       모멘트커피
Name: name, Length: 102, dtype: object


In [6]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(items) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    #카카오맵 접속 후 검색창에 입력
    driver.get("https://map.kakao.com/")
    
    #검색창에 식당명 입력   
    searchbox = driver.find_element(by='xpath', value="//input[@id='search.keyword.query']") 
    searchbox.send_keys(item)
    time.sleep(2)
    
  
    #검색 버튼 누르기
    searchbutton = driver.find_element(by='xpath', value="//button[@id='search.keyword.submit']") 
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    #검색한 식당이 있을 때 리뷰 정보 가져오기
    if len(driver.find_elements(by='xpath', value="//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        
        #리뷰수 버튼 클릭
        reviewbutton = driver.find_element(by='xpath', value="//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href")
        
        #리뷰창이 새 탭에서 뜨도록 처리 
        tabs = driver.window_handles
        driver.switch_to.window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(2)
        
        

        #스크롤 내리기
        for c in range(0,1):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        
        
        #리뷰 데이터 스크래핑을 위한 html 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        
        #리뷰 수가 0이 아닌 경우 리뷰글 가져오기
        if len(review_lists) != 0 :
            
            #후기 더보기가 있으면 모두 눌러주기 
            while True: 
                try:
                    if driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more").textContent') == "후기 더보기":
                        driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.ico_comm.ico_more").click()')
                        time.sleep(2) 
                        
                except:
                    print("더보기 종료")
                    break 
                
            #후기 더보기를 모두 누른 후 리뷰 리스트 재정의
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            review_lists = soup.select('.list_evaluation > li')
            print(len(review_lists))
                    
            
            #리뷰 정보가 있는 경우 리뷰글, 식당 평균 평점, 리뷰 작성 시간 가져오기     
            try:
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.comment_info > p > span') # 리뷰
                        rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text #식당평균평점
                        timestamp = review.select(' div > span.time_write') #시간정보
                        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            item,
                            rate,
                            user_review[0].text,
                            timestamp[0].text
                        ]
                        )            
    
                    time.sleep(1)
                
               
            except:
                print("더 이상 리뷰가 존재하지 않습니다.")
                
                
                
        else :
            #리뷰 정보가 없는 경우 식당명과 평균 평점만 데이터 프레임에 추가
            rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text
            rev_list.append(
                [
                    item,
                    rate,
                ]
            ) 

                    
            print("리뷰가 없습니다")
        
        #검색한 창 닫고 카카오맵 검색 페이지로 돌아가기    
        driver.close()
        driver.switch_to.window(tabs[0])
        print("기본 페이지로 돌아가기")
            
    else:
        print("식당이 존재하지 않음")
        
driver.close() 

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:01<00:00, 3.35MB/s]


진행상황 : 1/102
식당 존재
더보기 종료
15
기본 페이지로 돌아가기
진행상황 : 2/102
식당 존재
더보기 종료
28
기본 페이지로 돌아가기
진행상황 : 3/102
식당 존재
더보기 종료
47
기본 페이지로 돌아가기
진행상황 : 4/102
식당 존재
더보기 종료
24
기본 페이지로 돌아가기
진행상황 : 5/102
식당 존재
더보기 종료
88
기본 페이지로 돌아가기
진행상황 : 6/102
식당 존재
더보기 종료
26
기본 페이지로 돌아가기
진행상황 : 7/102
식당 존재
더보기 종료
19
기본 페이지로 돌아가기
진행상황 : 8/102
식당 존재
더보기 종료
23
기본 페이지로 돌아가기
진행상황 : 9/102
식당 존재
더보기 종료
26
기본 페이지로 돌아가기
진행상황 : 10/102
식당 존재
더보기 종료
34
기본 페이지로 돌아가기
진행상황 : 11/102
식당 존재
더보기 종료
122
기본 페이지로 돌아가기
진행상황 : 12/102
식당 존재
더보기 종료
134
기본 페이지로 돌아가기
진행상황 : 13/102
식당 존재
더보기 종료
104
기본 페이지로 돌아가기
진행상황 : 14/102
식당 존재
더보기 종료
11
기본 페이지로 돌아가기
진행상황 : 15/102
식당 존재
더보기 종료
27
기본 페이지로 돌아가기
진행상황 : 16/102
식당 존재
더보기 종료
96
기본 페이지로 돌아가기
진행상황 : 17/102
식당 존재
더보기 종료
46
기본 페이지로 돌아가기
진행상황 : 18/102
식당 존재
더보기 종료
46
기본 페이지로 돌아가기
진행상황 : 19/102
식당 존재
더보기 종료
124
기본 페이지로 돌아가기
진행상황 : 20/102
식당 존재
리뷰가 없습니다
기본 페이지로 돌아가기
진행상황 : 21/102
식당 존재
더보기 종료
43
기본 페이지로 돌아가기
진행상황 : 22/102
식당 존재
더보기 종료
61
기본 페이지로 돌아가기
진행상황 : 23/102
식당 존재
더보기 종료
33
기본 페이지로 돌아가기
진행상황 : 24/102
식당 

In [9]:
#스크래핑한 데이터를 데이터 프레임으로 만들기    
columns = ["name", "rate", "review", "timestamp"]
df = pd.DataFrame(rev_list, columns=columns)
df    

,name,rate,review,timestamp
0,마포비터스,4.4,칵테일…? 가격이 너무 터무니없어요. 그리고 20대 초반 애들만 바글거림..ㅜ 지하...,2022.10.05.
1,마포비터스,4.4,굿,2022.09.15.
2,마포비터스,4.4,<22년 1월 방문> 커스텀 칵테일도 재밌고 콘치즈.. 맛있어요,2022.09.12.
3,마포비터스,4.4,,2022.07.18.
4,마포비터스,4.4,내 아지트....,2022.07.07.
...,...,...,...,...
4652,마포 모멘트커피,3.0,캐릭터 확실 소품이랑 메뉴가 잘 어우러짐 맛은 쏘쏘,2019.04.04.
4653,마포 모멘트커피,3.0,남직원들 불친절해서 짜증났음다신 안 간다,2019.02.20.
4654,마포 모멘트커피,3.0,토요일 점심시간 쯔음 방문. 웨이팅이 (엄청나게) 길었지만 재밌고 맛있고 즐거운 시...,2018.11.03.
4655,마포 모멘트커피,3.0,특별히 좋을것도 나쁠것도 없었던 카페지만 일하는 직원의 응대가 조금 무성의하다고 느...,2018.09.10.


In [10]:
#데이터 프레임 전처리#

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

#중복값 제거
#df = df.drop_duplicates(['review'], keep='first',ignore_index = True)

#리뷰글이 없는 결측치 제거
df['review']=df['review'].dropna()

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

마포 타오마라탕    211
마포 피오니      192
마포 칸다소바     166
마포 청어람      151
마포 바다회사랑    151
           ... 
마포 코이크        1
마포 닝교초식당      1
마포 작당모의       1
마포 스윗헤르츠      1
마포 연어롭다       1
Name: name, Length: 101, dtype: int64
마포 타오마라탕    211
마포 피오니      192
마포 칸다소바     166
마포 청어람      151
마포 바다회사랑    151
           ... 
마포 코이크        1
마포 닝교초식당      1
마포 작당모의       1
마포 스윗헤르츠      1
마포 연어롭다       1
Name: name, Length: 101, dtype: int64


In [11]:
#스크래핑한 데이터 프레임을 csv로 저장
df.to_csv('마포구카카오맵리뷰.csv', encoding='utf-8-sig')